# ORF Density Notebook

- implementation for evolution necessary
- could also be necessary for better positional k-mers
- Status: finished
- no real knowledge gained!!
- discussing with manja again
- other idea:
    - Kozaki Sequence Probability (ORF Start/non-ORF Start)
    - Frameshift Probability (Start, Stop anderer Frame) 
    - All Stop/Start Positions plus Frame 
    - Prediction by prior knowledge 
        - eg. 3 proteins, 2 shifts and 1 alternative init (kozaki)
        - calculate best 3 and weight probabilitys by occurence of events
        - alternative splicing completely unimportant
        - only the protein frames with shifts and alternative inits important

![ORF](ORF.png)

In [235]:
from Bio import SeqIO
from Bio.Seq import Seq

In [630]:
def getReadingframe (seq):

    aminostr = {
        'ATG':'+', 'TAA':'-', 'TAG':'-', 'TGA':'-',
    }
    # aminorev = {
    #     'CAT':'-', 'TTA':'+', 'CTA':'+', 'TCA':'+',
    # }
    
    i = 0
    cs1 = cs2 = cs3 = len(seq)
    as1 = bs1 = fs1 = gs1 = as2 = bs2 = fs2 = gs2 = as3 = bs3 = fs3 = gs3 = 0
    # cr1 = cr2 = cr3 = len(seq)
    # ar1 = br1 = ar2 = br2 = ar3 = br3 = 0

    while i <= len(seq)-2:
        as1,bs1,cs1,fs1,gs1 = getORFDensity(aminostr.get(seq[i+0:i+3], ''),as1,bs1,cs1,fs1,gs1,i+0)
        as2,bs2,cs2,fs2,gs2 = getORFDensity(aminostr.get(seq[i+1:i+4], ''),as2,bs2,cs2,fs2,gs2,i+1)
        as3,bs3,cs3,fs3,gs3 = getORFDensity(aminostr.get(seq[i+2:i+5], ''),as3,bs3,cs3,fs3,gs3,i+2)
        # ar1,br1,cr1 = getORFDensity(aminorev.get(seq[i+0:i+3], ''),ar1,br1,cr1,i+0)
        # ar2,br2,cr2 = getORFDensity(aminorev.get(seq[i+1:i+4], ''),ar2,br2,cr2,i+1)
        # ar3,br3,cr3 = getORFDensity(aminorev.get(seq[i+2:i+5], ''),ar3,br3,cr3,i+2)

        i += 3
        
    # print(cs1, cs2, cs3, cr1, cr2, cr3)
    print(cs1, gs1, cs2, gs2, cs3, gs3)

In [671]:
def getORFDensity (acid,a,b,c,f,g,i):
    
    #a: Startcodon position
    #b: Stopcodon position
    #c: smallest Stop-Start distance
    #f: readingframe start
    #g: longest readingframe distance
    #i: position
    
    if acid == '+':
        a = i 
        f = i if f == 0 else f
        c = i-b if i-b <= c and b > 0 else c
        #c = c+i-b if b > 0 else c
          
    elif acid == '-':
        b = i
        g = i-f if i-f >= g and f > 0 else g
        f = 0 if f != 0 else f
    
    # print(a,b,c,f,g,i)
    return(a,b,c,f,g)

In [465]:
fasta = SeqIO.parse('A.fasta','fasta')

In [683]:
entry = next(fasta)
sequence =  str(entry.seq)
header = entry.name
print(header)
getReadingframe(sequence)

CY073729|A/Aalborg/INS283/2009|8|NS1|A|H1N1|12/07/2009|Human|Pass|09_10|Denmark|NA
45 657 42 324 3 42
